### 12.1.1 数据加载

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("./family_data.csv", index_col="family_id")
data

,choice_0,choice_1,choice_2,choice_3,choice_4,choice_5,choice_6,choice_7,choice_8,choice_9,n_people
family_id,,,,,,,,,,,
0,52,38,12,82,33,75,64,76,10,28,4
1,26,4,82,5,11,47,38,6,66,61,4
2,100,54,25,12,27,82,10,89,80,33,3
3,2,95,1,96,32,6,40,31,9,59,2
4,53,1,47,93,26,3,46,16,42,39,4
...,...,...,...,...,...,...,...,...,...,...,...
4995,16,1,66,33,18,70,56,46,86,60,4
4996,88,66,20,17,26,54,81,91,59,48,2
4997,32,66,54,17,27,21,74,81,3,7,6


### 12.1.2 数据预处理

+ 1）计算Perference Cost矩阵 pcost_mat
+ 2）计算Accounting Cost矩阵 acost_mat
+ 3）计算每个家庭的人数 FAMILY_SIZE
+ 4）每个家庭的倾向选择（choice_） DESIRED

#### 1 计算Perference Cost矩阵 pcost_mat

In [3]:
def get_penalty(n, choice):
    """
    n为家庭成员个数
    返回值：如果满足第choice个需求，需要的penalty
    """
    penalty = None
    if choice == 0:
        penalty = 0
    if choice == 1:
        penalty = 50
    if choice == 2:
        penalty = 50 + 9*n
    if choice == 3:
        penalty = 100 + 9*n
    if choice == 4:
        penalty = 200 + 9*n
    if choice == 5:
        penalty = 200 +18*n
    if choice == 6:
        penalty = 300 + 18*n
    if choice == 7:
        penalty = 300 + 36*n
    if choice == 8:
        penalty = 400 + 36*n
    if choice == 9:
        penalty = 500 + (36+199)*n
    if choice > 9:
        penalty = 500 + (36+398)*n
    return penalty

In [4]:
N_DAYS = 100 # 安排的天数
N_FAMILY = 5000 # 家庭ID的个数
MIN_OCCUPANCY = 125 # 最小承载量
MAX_OCCUPANCY = 300 # 最大承载量

In [5]:
# 计算pcost_mat,每个家庭，在什么时候(day 0-99)访问

In [6]:
# 初始化矩阵，shape为5000*100
# 每个家庭在某天去，给圣诞老人带来的成本
pcost_mat = np.full(shape=(N_FAMILY, 100), fill_value=999999) 

In [7]:
%%time
for f in range(N_FAMILY):
    f_num = data.loc[f, "n_people"]
    # 对于第f个家庭，初始化pocst= other choice下的Penalty,即最差的选择
    pcost_mat[f, :] = get_penalty(f_num, 10)
    # 计算choice0 - choice9的penalty
    for choice in range(10):
        temp = data.loc[f][choice] # choice的天数
        penalty = get_penalty(f_num, choice)
        pcost_mat[f, temp-1] = penalty
pcost_mat    

Wall time: 11.1 s


array([[2236, 2236, 2236, ..., 2236, 2236, 2236],
       [2236, 2236, 2236, ..., 2236, 2236, 2236],
       [1802, 1802, 1802, ..., 1802, 1802,    0],
       ...,
       [3104, 3104,  616, ..., 3104, 3104, 3104],
       [ 390, 2670, 2670, ..., 2670, 2670, 2670],
       [2236, 2236, 2236, ..., 2236, 2236, 2236]])

In [8]:
pcost_mat.shape

(5000, 100)

#### 2 计算Accounting Cost矩阵 acost_mat

只和前一天的参观人数和当天的参观人数有关

In [9]:
acost_mat = np.zeros(shape=(MAX_OCCUPANCY+1, MAX_OCCUPANCY+1), dtype= np.float64)

In [10]:
for i in range(acost_mat.shape[0]): # 当天安排的人数
    for j in range(acost_mat.shape[1]): # 前一天安排的人数
        diff = abs(i-j)
        acost_mat[i, j] = max(0, ((i-125) / 400) * i**(0.5 + diff/50.0))
acost_mat

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [4.16316072e+15, 3.71482922e+15, 3.31477861e+15, ...,
        7.46610759e+00, 8.36716954e+00, 9.37697794e+00],
       [4.79555148e+15, 4.27883100e+15, 3.81778713e+15, ...,
        8.43020770e+00, 7.52185316e+00, 8.43020770e+00],
       [5.52415954e+15, 4.92860244e+15, 4.39725208e+15, ...,
        9.51970597e+00, 8.49339085e+00, 7.57772228e+00]])

#### 3 计算每个家庭的人数 FAMILY_SIZE

In [11]:
FAMILY_SIZE = data["n_people"].values

In [12]:
FAMILY_SIZE

array([4, 4, 3, ..., 6, 5, 4], dtype=int64)

#### 4 每个家庭的倾向选择（choice_） DESIRED

In [13]:
DESIRED = data.values[:, :-1] - 1
DESIRED

array([[51, 37, 11, ..., 75,  9, 27],
       [25,  3, 81, ...,  5, 65, 60],
       [99, 53, 24, ..., 88, 79, 32],
       ...,
       [31, 65, 53, ..., 80,  2,  6],
       [66, 91,  3, ..., 11, 25, 69],
       [12, 10, 24, ..., 38, 17, 46]], dtype=int64)

### 12.1.3 使用LP和MIP求解 规划方案

In [14]:
from ortools.linear_solver import pywraplp

+ 1）先使用LP 对绝大部分家庭进行规划
+ 2）再使用MIP 对剩余家庭进行规划
+ 3）汇总两边的结果 => 最终规划方案

#### 1 先使用LP 对绝大部分家庭进行规划

In [15]:
# 假定自以preference_cost为目标
def solveLP():
    # 线性规划 优化器
    solver= pywraplp.Solver('AssignmentProblem', pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)
    x = {} # family_id 在第j天是否参观访问
    
    # 每一天有哪些家庭
    candidates = [[] for x in range(N_DAYS)]
    
    for i in range(N_FAMILY): # family_id
        for j in DESIRED[i, :]: # family_id 的choice
            candidates[j].append(i) # 在第j天，有第i个family参观
            # 定义决策变量x[i, j]，i代表family_id, j代表第j天参观
            x[i, j] =  solver.BoolVar("x[%i, %i]"%(i, j))
            
    # 每天参观的人数
    daily_occupancy = [solver.Sum(x[i, j] * FAMILY_SIZE[i] for i in candidates[j])
                       for j in range(N_DAYS)] # j代表1-100天
    # 每个家庭，在10个choice中出现的总数
    family_presence = [solver.Sum(x[i, j] for j in DESIRED[i, :])
                      for i in range(N_FAMILY)]
    # 定义目标函数preference cost部分
    preference_cost = solver.Sum(pcost_mat[i, j]*x[i,j] for i in range(N_FAMILY) \
                                for j in DESIRED[i, :])
    # 满足preference_cost最小
    solver.Minimize(preference_cost)
    
    # 人为增加约束条件Constraints
    for j in range(N_DAYS-1):
        # 当前人数不超过前一天人数+25
        solver.Add(daily_occupancy[j] -daily_occupancy[j+1] <= 25)
#         solver.Add(daily_occupancy[j+1] -daily_occupancy[j] <= 25)

    # 每个家庭都在10个choice中出现一次
    for i in range(N_FAMILY):
        solver.Add(family_presence[i] == 1)

    # 每天访问人数约束
    for j in range(N_DAYS):
        solver.Add(daily_occupancy[j] >= MIN_OCCUPANCY)
        solver.Add(daily_occupancy[j] <= MAX_OCCUPANCY)
    result = solver.Solve()
    
    print(solver.Objective().Value())
    # result
    temp = [(i, j, x[i,j].solution_value()) for i in range(N_FAMILY)\
            for j in DESIRED[i, :] if x[i, j].solution_value() > 0 ]
#     print(temp)
    # 得到参观日期的安排
    df = pd.DataFrame(temp, columns=["family_id", "day", "result"])
    
#     return result
    return df
    

In [16]:
%%time
result = solveLP()
result

65827.15476190476
Wall time: 16 s


,family_id,day,result
0,0,51,1.0
1,1,25,1.0
2,2,99,1.0
3,3,1,1.0
4,4,52,1.0
...,...,...,...
5074,4995,15,1.0
5075,4996,87,1.0
5076,4997,31,1.0
5077,4998,91,1.0


LP:15.9s

一共只有5000个家庭，确有5079行，说明重复了

In [17]:
result["family_id"].value_counts()

3394    2
481     2
1770    2
4775    2
4419    2
       ..
1202    1
3251    1
1206    1
3255    1
0       1
Name: family_id, Length: 5000, dtype: int64

In [18]:
# 设置阈值
THRS = 0.999
# 已将安排上的
assigned_df = result[result.result >= THRS]

In [19]:
assigned_df

,family_id,day,result
0,0,51,1.0
1,1,25,1.0
2,2,99,1.0
3,3,1,1.0
4,4,52,1.0
...,...,...,...
5074,4995,15,1.0
5075,4996,87,1.0
5076,4997,31,1.0
5077,4998,91,1.0


In [20]:
# 没有安排上的家庭(不为0和1)
unassigned_df = result[(result.result < THRS) & (result.result > (1 - THRS))]
unassigned_df

,family_id,day,result
123,123,94,0.250000
124,123,21,0.750000
141,140,60,0.666667
142,140,12,0.333333
327,322,81,0.500000
...,...,...,...
4926,4850,34,0.750000
5029,4952,65,0.400000
5030,4952,17,0.600000
5057,4979,13,0.500000


In [21]:
# 计算每天访问的人数(根据assigned_df)
assigned_df["family_size"] = FAMILY_SIZE[assigned_df.family_id]
assigned_df

C:\Users\LGB\Anaconda3\envs\lgb_02\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,family_id,day,result,family_size
0,0,51,1.0,4
1,1,25,1.0,4
2,2,99,1.0,3
3,3,1,1.0,2
4,4,52,1.0,4
...,...,...,...,...
5074,4995,15,1.0,4
5075,4996,87,1.0,2
5076,4997,31,1.0,6
5077,4998,91,1.0,5


In [22]:
# 按照day进行聚合
occupancy = assigned_df.groupby("day").family_size.sum().values
occupancy

array([295, 271, 297, 297, 269, 248, 222, 198, 196, 297, 296, 293, 268,
       246, 220, 239, 296, 285, 269, 244, 219, 186, 208, 299, 300, 296,
       271, 238, 212, 195, 295, 275, 250, 221, 195, 172, 148, 293, 271,
       247, 221, 196, 175, 149, 297, 272, 244, 222, 196, 173, 190, 281,
       256, 236, 206, 180, 159, 174, 233, 210, 186, 163, 140, 125, 115,
       223, 198, 165, 140, 125, 125, 122, 217, 195, 173, 147, 124, 122,
       119, 218, 200, 173, 138, 120, 125, 122, 246, 225, 192, 173, 150,
       125, 120, 223, 199, 173, 150, 125, 125, 125], dtype=int64)

In [23]:
# 更新每日人数上下限
min_occupancy = np.array([max(0, MIN_OCCUPANCY-x) for x in occupancy])
min_occupancy

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 10,  0,  0,  0,
        0,  0,  0,  3,  0,  0,  0,  0,  1,  3,  6,  0,  0,  0,  0,  5,  0,
        3,  0,  0,  0,  0,  0,  0,  5,  0,  0,  0,  0,  0,  0,  0],
      dtype=int64)

In [24]:
max_occupancy = np.array([MAX_OCCUPANCY-x for x in occupancy])
max_occupancy

array([  5,  29,   3,   3,  31,  52,  78, 102, 104,   3,   4,   7,  32,
        54,  80,  61,   4,  15,  31,  56,  81, 114,  92,   1,   0,   4,
        29,  62,  88, 105,   5,  25,  50,  79, 105, 128, 152,   7,  29,
        53,  79, 104, 125, 151,   3,  28,  56,  78, 104, 127, 110,  19,
        44,  64,  94, 120, 141, 126,  67,  90, 114, 137, 160, 175, 185,
        77, 102, 135, 160, 175, 175, 178,  83, 105, 127, 153, 176, 178,
       181,  82, 100, 127, 162, 180, 175, 178,  54,  75, 108, 127, 150,
       175, 180,  77, 101, 127, 150, 175, 175, 175], dtype=int64)

#### 2 使用整数规划进行求解

In [25]:
def solveIP(families, min_occupancy, max_occupancy):
    # 创建求解器
    solver= pywraplp.Solver('AssignmentProblem', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
    # 需要安排的家庭
    n_familes = len(families)
    
    x = {} # family_id 在第j天是否参观访问
    
    # 每一天有哪些家庭
    candidates = [[] for x in range(N_DAYS)]
    
    for i in families: # family_id
#         print(i)
        for j in DESIRED[i, :]: # family_id 的choice
            candidates[j].append(i) # 在第j天，有第i个family参观
            # 定义决策变量x[i, j]，i代表family_id, j代表第j天参观
            x[i, j] =  solver.BoolVar("x[%i, %i]"%(i, j))
            
    # 每天参观的人数
    daily_occupancy = [solver.Sum(x[i, j] * FAMILY_SIZE[i] for i in candidates[j])
                       for j in range(N_DAYS)] # j代表1-100天
    # 每个家庭，在10个choice中出现的总数
    family_presence = [solver.Sum(x[i, j] for j in DESIRED[i, :])
                      for i in families]
    # 定义目标函数preference cost部分
    preference_cost = solver.Sum(pcost_mat[i, j]*x[i,j] for i in families \
                                for j in DESIRED[i, :])
    # 满足preference_cost最小
    solver.Minimize(preference_cost)
    
    # 人为增加约束条件Constraints
    for j in range(N_DAYS-1):
        # 当前人数不超过前一天人数+25
        solver.Add(daily_occupancy[j] -daily_occupancy[j+1] <= 25)
#         solver.Add(daily_occupancy[j+1] -daily_occupancy[j] <= 25)

    # 每个家庭都在10个choice中出现一次
    for i in range(n_familes):
        solver.Add(family_presence[i] == 1)

    # 每天访问人数约束
    for j in range(N_DAYS):
        solver.Add(daily_occupancy[j] >= min_occupancy[j])
        solver.Add(daily_occupancy[j] <= max_occupancy[j])
    result = solver.Solve()
    
    print(solver.Objective().Value())
    # result
    temp = [(i, j, x[i,j].solution_value()) for i in families\
            for j in DESIRED[i, :] if x[i, j].solution_value() > 0 ]
#     print(temp)
    # 得到参观日期的安排
    df = pd.DataFrame(temp, columns=["family_id", "day", "result"])
    
#     return result
    return df
    

In [26]:
unassigned = unassigned_df.family_id.unique()
unassigned

array([ 123,  140,  322,  380,  427,  455,  481,  541,  620,  635,  686,
        798,  873,  966, 1016, 1114, 1120, 1193, 1436, 1548, 1611, 1808,
       1915, 2004, 2019, 2054, 2465, 2474, 2621, 2680, 2699, 3033, 3054,
       3326, 3394, 3424, 3473, 3537, 3620, 3749, 3953, 3977, 4055, 4057,
       4065, 4139, 4156, 4199, 4284, 4376, 4419, 4561, 4667, 4698, 4768,
       4837, 4850, 4952, 4979], dtype=int64)

In [27]:
%%time
result = solveIP(unassigned, min_occupancy, max_occupancy)
result

1701.0
Wall time: 141 ms


,family_id,day,result
0,123,94,1.0
1,140,60,1.0
2,322,81,1.0
3,380,72,1.0
4,427,41,1.0
5,455,0,1.0
6,481,71,1.0
7,541,59,1.0
8,620,78,1.0
9,635,83,1.0


#### 3 汇总两边的结果

In [28]:
df = pd.concat((assigned_df[["family_id", "day"]], result[["family_id", "day"]])).sort_values("family_id")
df

,family_id,day
0,0,51
1,1,25
2,2,99
3,3,1
4,4,52
...,...,...
5074,4995,15
5075,4996,87
5076,4997,31
5077,4998,91


In [29]:
from numba import njit

In [30]:
# 根据安排情况，计算这个安排的preference cost
@njit(fastmath=True)
def pcost(prediction):
    daily_occupancy = np.zeros(N_DAYS+1, dtype=np.int64)
    penalty = 0
    for i,p in enumerate(prediction):
        # 计算家庭人数
        n = FAMILY_SIZE[i]
        # 第i个家庭，p天访问时的cost
        penalty += pcost_mat[i, p]
        # 计算当天的人数
        daily_occupancy[p] += n
    return penalty, daily_occupancy

In [31]:
# 根据安排情况，计算安排的accounting cost
@njit(fastmath=True)
def acost(daily_occupancy):
    accounting_cost = 0
    num_out_of_range = 0
    for day in range(N_DAYS):
        n_p1 = daily_occupancy[day+1] # 前一天
        n = daily_occupancy[day]
        # 如果超过了承载范围，则设置out_of_range
        num_out_of_range += (n>MAX_OCCUPANCY) or (n<MIN_OCCUPANCY)
        # 计算accounting cost
        accounting_cost += acost_mat[n, n_p1]
    return accounting_cost, num_out_of_range

In [32]:
@njit(fastmath=True)
def cost_function(prediction):
    # 基于prediction，计算perference cost和accounting cost
    penalty, daily_occupancy = pcost(prediction) # 统计perference cost和每天承载数量
    accounting_cost, num_out_of_range = acost(daily_occupancy) # 根据每天承载数量，计算accouting cost
    final_score = penalty + accounting_cost + num_out_of_range*99999
    return final_score

In [33]:
prediction = df.day.values
prediction

array([51, 25, 99, ..., 31, 91, 12], dtype=int64)

In [34]:
cost_function(prediction)

3326405.0493958425

In [35]:
def save_result(pred, filename):
    result = pd.DataFrame(range(N_FAMILY), columns=["family_id"])
    result["assigned_day"] = pred + 1
    result.to_csv(filename, index=False)
    print(filename+" saved!")
    return result

In [36]:
result = save_result(prediction, "submission1.csv")

submission1.csv saved!


In [37]:
result

,family_id,assigned_day
0,0,52
1,1,26
2,2,100
3,3,2
4,4,53
...,...,...
4995,4995,16
4996,4996,88
4997,4997,32
4998,4998,92


### 12.1.4 寻找更好的替代方案

In [38]:
def find_better(pred):
    fobs = np.argsort(FAMILY_SIZE)
#     print(fobs)
    score = cost_function(pred)
    origianl_score = np.inf
    
    # 如果找不到更新，则退出
    while score < origianl_score:
        origianl_score = score
        for family_id in fobs:
            for pick in range(10):
                # 得到family_id在choice pick的日期
                day = DESIRED[family_id, pick]
                # 该family的原有日期old value
                oldvalue = pred[family_id]
                pred[family_id] = day
                # 重新计算分数
                new_score = cost_function(pred)
                # 如果比原来分数小，更新choice成功
                if new_score < score:
                    score = new_score
                else:
                    pred[family_id] = oldvalue
        print(score, end="\r")
    print(score)

In [39]:
new = prediction.copy()
find_better(new)

73218.384383220526


In [40]:
cost_function(new)

73218.38438322052

In [41]:
save_result(new, "submission2.csv")

submission2.csv saved!


,family_id,assigned_day
0,0,52
1,1,26
2,2,100
3,3,2
4,4,53
...,...,...
4995,4995,16
4996,4996,88
4997,4997,32
4998,4998,92
